In [2]:
!featurize dataset download 95bda819-7a90-4f5a-973b-414c5bbf0fb3

  🚀 开始下载数据集 MiniImageNet 至 ./
  ✅ 下载完成 [#######################################>] 2.88 GiB/2.89 GiB (0.0s)


In [7]:
import os  
import shutil  

# 定义源文件和目标文件路径  
src_root = "miniimagenet/images"  
dst_dir = "dataset/n02101006"  

# 创建目标文件夹（如果不存在的话）  
os.makedirs(dst_dir, exist_ok=True)  

# 初始化计数器  
count = 0  

# 遍历源文件夹及其子文件夹  
for root, dirs, files in os.walk(src_root):  
    for file in files:  
        if file.lower().endswith(".jpg"):  
            # 判断文件名是否以指定前缀开头  
            if file.startswith("n02101006"):  
                src_file = os.path.join(root, file)  
                dst_file = os.path.join(dst_dir, file)  
                try:  
                    # 复制文件  
                    shutil.copy2(src_file, dst_file)  
                    print(f"已复制文件：{file}")  
                    count += 1  
                    # 当复制到30张时停止  
                    if count >= 100:  
                        print(f"已复制30张图片，任务完成！")  
                        # 需要break两次循环：先break内层循环，然后break外层循环  
                        break  
                except Exception as e:  
                    print(f"复制文件 {file} 时出现错误：{str(e)}")  
            if count >= 30:  
                break  
    if count >= 30:  
        break  

已复制文件：n0210100600000642.jpg
已复制文件：n0210100600000134.jpg
已复制文件：n0210100600000693.jpg
已复制文件：n0210100600000927.jpg
已复制文件：n0210100600000529.jpg
已复制文件：n0210100600000010.jpg
已复制文件：n0210100600000803.jpg
已复制文件：n0210100600001231.jpg
已复制文件：n0210100600001126.jpg
已复制文件：n0210100600000639.jpg
已复制文件：n0210100600000017.jpg
已复制文件：n0210100600000300.jpg
已复制文件：n0210100600001121.jpg
已复制文件：n0210100600000761.jpg
已复制文件：n0210100600001005.jpg
已复制文件：n0210100600000920.jpg
已复制文件：n0210100600000983.jpg
已复制文件：n0210100600000190.jpg
已复制文件：n0210100600000141.jpg
已复制文件：n0210100600000520.jpg
已复制文件：n0210100600001238.jpg
已复制文件：n0210100600001182.jpg
已复制文件：n0210100600000065.jpg
已复制文件：n0210100600000876.jpg
已复制文件：n0210100600000404.jpg
已复制文件：n0210100600001295.jpg
已复制文件：n0210100600000372.jpg
已复制文件：n0210100600001153.jpg
已复制文件：n0210100600000062.jpg
已复制文件：n0210100600001243.jpg


In [1]:
import pandas as pd  

def select_files():  
    try:  
        # 读取train.csv文件  
        train_df = pd.read_csv('miniimagenet/train.csv')  
        
        # 确保'filename'列存在  
        if 'filename' not in train_df.columns:  
            print("错误：train.csv文件中没有'filename'列。")  
            return  
        
        # 定义要筛选的前缀列表  
        prefixes = ['n02105505','n02101006']  
         
        
        # 初始化结果字典，用于存储每个前缀对应的文件名列表  
        selected_files = {f'files_startswith_{prefix}': [] for prefix in prefixes}  
        
        # 遍历每一行，检查filename是否以指定的前缀开头且以.jpg结尾  
        for index, row in train_df.iterrows():  
            filename = row['filename']  
            if isinstance(filename, str):  # 确保filename是字符串  
                if filename.lower().endswith('.jpg'):  
                    for prefix in prefixes:  
                        if filename.startswith(prefix):  
                            selected_files[f'files_startswith_{prefix}'].append(filename)  
        
        # 创建结果DataFrame  
        # 需要确保每列的长度一致，这里假设每列至少有一个文件名  
        # 如果某列没有文件名，则会显示为空  
        data = selected_files  
        
        # 创建DataFrame时，每列为一个列表，确保每列长度相同  
        max_length = max(len(v) for v in data.values())  
        dataframe_rows = []  
        for i in range(max_length):  
            row = {}  
            for key in data:  
                if i < len(data[key]):  
                    row[key] = data[key][i]  
                else:  
                    row[key] = ''  # 如果某列没有对应的行，设置为空字符串  
            dataframe_rows.append(row)  
        
        result_df = pd.DataFrame(dataframe_rows)  
        
        # 保存到CSV文件，不写入索引  
        result_df.to_csv('selected_files.csv', index=False)  
        print("已将符合条件的文件保存到selected_files.csv文件中。")  
    
    except FileNotFoundError:  
        print("错误：train.csv文件未找到。请确保文件位于当前工作目录中。")  
    except pd.errors.EmptyDataError:  
        print("错误：train.csv文件为空。")  
    except pd.errors.ParserError:  
        print("错误：读取train.csv文件时发生错误。请检查文件格式是否正确。")  
    except Exception as e:  
        print(f"未知错误：{str(e)}")  



In [2]:
select_files()

已将符合条件的文件保存到selected_files.csv文件中。


In [7]:
import pandas as pd  
import os  
import shutil  

def copy_images_to_dataset(target_prefix = 'n02105505'):  
    try:  
        # 读取selected_files.csv文件  
        selected_df = pd.read_csv('selected_files.csv')  
        
        # 定义需要处理的前缀，这里以“n02105505”为例  
        target_folder = f'dataset/{target_prefix}'  
        
        # 确保目标文件夹存在，若不存在则创建  
        os.makedirs(target_folder, exist_ok=True)  
        
        # 获取当前列名中包含目标前缀的列  
        target_column = f'files_startswith_{target_prefix}'  
        
        # 检查目标列是否存在  
        if target_column not in selected_df.columns:  
            print(f"错误：selected_files.csv文件中没有'{target_column}'列。")  
            return  
        
        # 获取所有的filename，排除空值  
        filenames = selected_df[target_column].dropna()  
        
        # 定义图片所在的源文件夹  
        source_folder = 'miniimagenet/images'  
        
        # 遍历每一个filename，并复制到目标文件夹  
        for filename in filenames:  
            if pd.isna(filename):  
                continue  # 跳过空值  
                
            # 构建完整的源文件路径  
            src_file_path = os.path.join(source_folder, filename)  
            
            # 检查文件是否存在  
            if os.path.isfile(src_file_path):  
                # 构建目标文件路径  
                dst_file_path = os.path.join(target_folder, filename)  
                # 复制文件  
                shutil.copy2(src_file_path, dst_file_path)  
                # print(f"已成功复制文件：{filename}")  
            else:  
                print(f"警告：文件 {filename} 不存在。")  
        
        print(f"所有以 '{target_prefix}' 开头的图片已复制到 '{target_folder}' 文件夹中。")  
    
    except FileNotFoundError:  
        print("错误：selected_files.csv 文件未找到。")  
    except Exception as e:  
        print(f"未知错误：{str(e)}")  


In [8]:
 copy_images_to_dataset(target_prefix = 'n02105505') 

所有以 'n02105505' 开头的图片已复制到 'dataset/n02105505' 文件夹中。


In [9]:
 copy_images_to_dataset(target_prefix = 'n02101006') 

所有以 'n02101006' 开头的图片已复制到 'dataset/n02101006' 文件夹中。


In [6]:
import os  
import shutil  

def delete_all_in_dir(directory_path):  
    """  
    删除指定目录下的所有文件和子目录。  
    """  
    # 检查路径是否存在  
    if not os.path.exists(directory_path):  
        print(f"路径 {directory_path} 不存在。")  
        return  
    # 检查是否是目录  
    if not os.path.isdir(directory_path):  
        print(f"{directory_path} 不是目录。")  
        return  
    
    # 获取目录中的所有内容  
    for item in os.listdir(directory_path):  
        item_path = os.path.join(directory_path, item)  
        try:  
            # 如果是文件，直接删除  
            if os.path.isfile(item_path):  
                os.remove(item_path)  
                # print(f"已删除文件：{item_path}")  
            # 如果是目录，递归删除  
            elif os.path.isdir(item_path):  
                shutil.rmtree(item_path)  
                print(f"已删除目录：{item_path}")  
        except Exception as e:  
            print(f"无法删除 {item_path}，错误信息：{str(e)}")  
    
    print(f"已清空目录：{directory_path}")  

# 示例用法，替换为你要清空的目录路径  
directory_path = "dataset/n02101006"  
delete_all_in_dir(directory_path) 

已清空目录：dataset/n02101006
